In [1]:
%run Futures.ipynb
%run Options.ipynb

In [2]:
import pandas as pd
import numpy as np
from datetime import date, time, datetime
from py_vollib.black_scholes_merton.implied_volatility import implied_volatility as iv
from math import exp, sqrt, log, fabs
import matplotlib.pyplot as plt
from itertools import repeat
import os
import calendar 
import glob

In [3]:
work_dir = os.getcwd()
work_dir
Option_path = os.path.join(work_dir, 'Data', 'Option')
Future_path = os.path.join(work_dir, 'Data', 'Futures')
OptionOpen_path = os.path.join(work_dir, 'Data', 'OptionOpen')
OptionClose_path = os.path.join(work_dir, 'Data', 'OptionClose')
Rate_path = os.path.join(work_dir, 'Data', 'Rate')

In [4]:
Option_path

'C:\\Users\\Evan\\Documents\\GitHub\\TXO_TX_Strategy\\Data\\Option'

## Calculate the Date Difference

In [54]:
def dates_delta(start_time, maturity):
    '''
        Compute the distance between two TRADE_DATEs
        Already consider about the weekly options
    '''
    
    s = start_time
    if ~isinstance(maturity, str): 
        maturity = str(int(maturity))
        
    d0 = date(year = int(s/10000), month = int((s % 10000)/100), day = int(s % 100))
    if maturity[-2] == 'W':
        m = maturity[:5]
        year = int(int(maturity[:6])/100)
        month = int(int(maturity[:6]) % 100)
        weekidx = int(maturity[-1]) - 1
        d1 = get_expireDate(year, month, week = weekidx)
        
    else: 
        year = int(int(maturity[:6])/100)
        month = int(int(maturity[:6]) % 100)
        d1 = get_expireDate(year, month)

    t = np.busday_count(d0, d1)/252
    if t < 0:
        print("The input dates error!")
    return t

In [6]:
def trans_dt(d):
    d_new = date(year = int(d/10000), month = int((d % 10000)/100), day = int(d % 100))
    return d_new

In [7]:
def get_expireDate(year, month, week = 2):
    c = calendar.Calendar(firstweekday=calendar.SATURDAY)
    monthcal = c.monthdatescalendar(year, month)
    monthly_expire_date = monthcal[week][-3]
    return monthly_expire_date

In [8]:
def get_Future_List(): 
    extension = 'csv'
    os.chdir(Future_path)
    Fut_csv_List = glob.glob('*.{}'.format(extension))
    return Fut_csv_List

In [9]:
def get_Option_List(): 
    extension = 'csv'
    os.chdir(Option_path)
    Opt_csv_List = glob.glob('*.{}'.format(extension))
    return Opt_csv_List

In [10]:
def get_zcb_rate(transDate):
    if ~isinstance(transDate, date): 
        transDate = np.datetime64(trans_dt(transDate))
    
    rate =  pd.read_csv(os.path.join(Rate_path, 'ZCB.csv'))
    rate['Date'] = pd.to_datetime(rate['Date'])
    zcb = float(rate[rate.Date == transDate]['ZCB'])
    
    return zcb

In [11]:
def MK_disc(S, K, t): 
    'Maturity and Moneyness Discount'
    t = t * 252 # transform into days
    try:
        
        m = float(K/S - 1) # moneyness
        M = max(1, t/30.0) # days to month; at least one-month
        w = exp(-(m**2)/2 - (M - 1)**2)
    except ZeroDivisionError:
        M = max(1, t/30.0)
        w = exp(- (M - 1)**2)
    return w

In [41]:
def call_put_pair(Opt, Fut):
    '''
        Pair the Put Call Options
    '''
    pair_list = []
    pair_dic = {}
    
    length = Opt.get_length()
    
    for i in range(length):
        
        transDate= Opt.get_transDate()
        maturity = Opt.get_maturity(i)
        t = dates_delta(transDate , maturity) 
        K = Opt.get_excercisePrice(i)
        right = Opt.get_right(i)
        interval = Opt.get_interval()
        
        dict_key = "K:" + str(K) + "/" + "T:" + str(t)
        
        S = float(Fut.get_TransPrice(transTime))
        price = Opt.get_transPrice(i)
        
        cur_vol = Opt.get_volumn(i) * MK_disc(S, K, t) # current vol

#=============================================================================
#   Transform the Dict into Pair            
#=============================================================================

        if dict_key not in pair_dic:

            pair_dic[dict_key] = len(pair_dic)
            if right == "C":
                pair_list.append([Opt.df.iloc[i], [], cur_vol, S, 0])
            if right == "P":
                pair_list.append([[], Opt.df.iloc[i], cur_vol, 0, S])
                
        else:
            if right == "C":
                if len(pair_list[pair_dic[dict_key]][0]) < 1:
                    pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                    pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    pair_list[pair_dic[dict_key]][3] = S
                    
                if interval == "Openinterval": 

                    if pair_list[pair_dic[dict_key]][0]['TransTime'] > Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][3] = S
                        
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        
                elif interval == "Closeinterval": 
                    if pair_list[pair_dic[dict_key]][0]['TransTime'] < Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][0] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][3] = S
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                
            if right == "P":
                if len(pair_list[pair_dic[dict_key]][1]) < 1:
                    pair_list[pair_dic[dict_key]][1] = df.iloc[i]
                    pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                    pair_list[pair_dic[dict_key]][4] = S
                    
                if interval == "Openinterval": 
                    if pair_list[pair_dic[dict_key]][1]['TransTime'] > Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][1] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][4] = S
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        
                elif interval == "Closeinterval": 
                    if pair_list[pair_dic[dict_key]][1]['TransTime'] < Opt.df['TransTime'][i]:
                        pair_list[pair_dic[dict_key]][1] = Opt.df.iloc[i]
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
                        pair_list[pair_dic[dict_key]][4] = S
                    else:
                        pair_list[pair_dic[dict_key]][2] = pair_list[pair_dic[dict_key]][2] + cur_vol
    
    return pair_list

In [68]:
def volatility_spread_hour(Opt, Fut):
    volatility_spread = []
    spread_volume = []
    pair_list = call_put_pair(Opt, Fut)
    
    for i in range(len(pair_list)):
        
        if len(pair_list[i][0]) > 1 and len(pair_list[i][1]) > 1: # pair exist
            Sc = float(pair_list[i][3])
            Sp = float(pair_list[i][4])
            Sa = (Sc+Sp)/2.0
            
            transDate = Opt.get_transDate()
            maturity = pair_list[i][0]['Maturity']
            t = dates_delta(transDate , maturity)
            
            r = get_zcb_rate(transDate)
            K = float(pair_list[i][0]['ExcercisePrice'])
            q = 0
            moneyness = float(K/Sa)
            
            
            call_price = float(pair_list[i][0]['TransPrice'])
            put_price = float(pair_list[i][1]['TransPrice'])
            vol = pair_list[i][2]
            
            PV_K = K*exp(-r*t)
            intrinsic_c = fabs(max(Sc - PV_K, 0.0))
            intrinsic_p = fabs(max(PV_K - Sp, 0.0))
                
            if call_price < intrinsic_c or call_price >= S: #Out of the Boundary
                volatility_spread.append(0.0)
                spread_volume.append(0.0)
    
            elif put_price < intrinsic_p or put_price >= K:
                volatility_spread.append(0.0)
                spread_volume.append(0.0)
                
            elif Sc == 0 or Sp == 0: #typo error
                volatility_spread.append(0.0)
                spread_volume.append(0.0) 
                
            elif moneyness > 1.1 or moneyness < 0.9:
                volatility_spread.append(0.0)
                spread_volume.append(0.0) 
                
            else:
                call_iv = iv(price = call_price, 
                             flag = 'c', 
                             S = Sa, 
                             K = K, 
                             t = t, 
                             r = r,
                             q = q)
                
                put_iv = iv(price = put_price, 
                             flag = 'p', 
                             S = Sa, 
                             K = K, 
                             t = t, 
                             r = r,
                             q = q)
                
                cpiv = call_iv - put_iv
                print("cpiv:" ,cpiv)
                print("vol:" , vol)
                print("=====================")
                volatility_spread.append(cpiv)
                spread_volume.append(vol)
                
        else:
            volatility_spread.append(0.0)
            spread_volume.append(0.0)
            
    try: 
        weights_aggr = [i/sum(spread_volume) for i in spread_volume]
        hour_vs = np.average(volatility_spread, weights = weights_aggr)
        
    except ZeroDivisionError:
        hour_vs = np.nan

    return hour_vs

In [ ]:
def saving_file(df, saving_name = "df", results_path = os.path.join(work_dir, "Output_Result")):
    if not os.path.isdir(results_path):
        os.mkdir(results_path)
    results_path = os.path.join(results_path, datetime.today().strftime('%Y%m%d')) 
    print ("writing file to {}".format(results_path))

    if os.path.exists(results_path):
        results_path = results_path + '_' + datetime.datetime.today().strftime('_%H%M%S')
    if not os.path.isdir(results_path):
        os.mkdir(results_path)
        
    if df:
        df.to_csv(os.path.join(results_path, saving_name))
    else:
        print("Saving Files Failure!")

In [37]:
if __name__ ==  '__main__':
    Opt_csv_List = get_Option_List()
    dataCsv = Opt_csv_List[0]
    path = OptionOpen_path
    Opt = Options(path, dataCsv)
    Fut_csv_list = get_Future_List()
    FutCsv = Fut_csv_list[0]
    Fut = Futures(FutCsv)

In [42]:
pair_list = call_put_pair(Opt, Fut)

In [57]:
pair_list[50][0]

TransDate             20200313
Product               TXO     
ExcercisePrice           10500
Maturity                202003
Right                   C     
TransTime                84500
TransPrice                  25
Volumn                      10
Name: 12771, dtype: object

In [56]:
i = 50
transDate = Opt.get_transDate()
maturity = pair_list[i][0]['Maturity']
r = get_zcb_rate(transDate)

K = float(pair_list[i][0]['ExcercisePrice'])

t = dates_delta(transDate , maturity)


0.7656
202003.0


0.011904761904761904

In [69]:
hour_vs = volatility_spread_hour(Opt, Fut)

cpiv: -0.05878808116742329
vol: 1180.3810340026178
cpiv: -0.216146555904846
vol: 1353.6697924814964
cpiv: -0.3812547498482399
vol: 1192.917050248515
cpiv: -0.3597548278211767
vol: 2201.997526775424
cpiv: -0.1975597743729821
vol: 2017.9219525034125
cpiv: -0.21737420501483307
vol: 1637.7015958167501
cpiv: -0.22565195675827177
vol: 1785.229201037426
cpiv: -0.3368008167732671
vol: 2439.079412177586
cpiv: -0.2538194156338549
vol: 2156.306131382686
cpiv: -0.2408516586383478
vol: 707.9238772863197
cpiv: -0.45266085785703253
vol: 2592.2884238638762
cpiv: -0.49336125630007627
vol: 2345.1551302324333
cpiv: -0.6529807944070183
vol: 8.977624913692772
cpiv: -0.5215057254023503
vol: 3134.737318303346
cpiv: -0.6443688235458715
vol: 34.88578891593561
cpiv: -0.22788140709619137
vol: 1958.846488225511
cpiv: -0.6615981223162444
vol: 162.3243404070956


In [70]:
hour_vs

-0.3289851441896803